In [ ]:
import pickle
import boris_extraction as boris
import multirecording_spikeanalysis as spike
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.stats import sem
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel
import matplotlib.patches as mpatches
from itertools import combinations

def hex_2_rgb(hex_color): # Orange color
    rgb_color = tuple(int(hex_color[i:i+2], 16) / 255.0 for i in (1, 3, 5))
    return rgb_color

def pickle_this(thing_to_pickle, file_name):
    """
    Pickles things
    Args (2):   
        thing_to_pickle: anything you want to pickle
        file_name: str, filename that ends with .pkl 
    Returns:
        none
    """
    with open(file_name,'wb') as file:
        pickle.dump(thing_to_pickle, file)

def unpickle_this(pickle_file):
    """
    Unpickles things
    Args (1):   
        file_name: str, pickle filename that already exists and ends with .pkl
    Returns:
        pickled item
    """
    with open(pickle_file, 'rb') as file:
        return(pickle.load(file))

In [ ]:
phase2_collection = unpickle_this('phase2collection.pkl')

pca_analysis = spike.SpikeAnalysis_MultiRecording(phase2_collection, timebin = 100,ignore_freq =  0.5)

In [ ]:
pca_df, coeff, var = pca_analysis.PCA_trajectories(2, 2, d = 3, events = ['cagemate', 'recall', 'novel', 'acquisition'])
geo_dis = pca_analysis.LOO_PCA(2, 2, 0.9, events = ['acquisition', 'recall', 'cagemate', 'novel'])
var[0:3]
total_var = 0
for varl in var[0:10]: 
    total_var =+ total_var + varl
    print(total_var)


In [ ]:
pca_df, coeff, var = pca_analysis.PCA_trajectories(5, 5, d = 2, events = ['novel', 'acquisition'])

pca_df, coeff, var = pca_analysis.PCA_trajectories(5, 5, d = 2, events = ['recall', 'acquisition'])

In [ ]:
pca_df, coeff, var = pca_analysis.PCA_trajectories(2, 2, d = 3, events = ['exposure 1', 'exposure 2', 'exposure 3', 'acquisition'])
pca_df, coeff, var = pca_analysis.PCA_trajectories(2, 2, events = ['exposure 1', 'exposure 2', 'exposure 3', 'acquisition'])
order_distances = pca_analysis.avg_geo_dist(2, 2, .9, post_window = 0, events =['exposure 1', 'exposure 2', 'exposure 3', 'acquisition'])

# Plotting

In [ ]:
pca_analysis = spike.SpikeAnalysis_MultiRecording(phase2_collection, 100,50,0.5)
pca_df, coeff, var = pca_analysis.PCA_trajectories(2, 2, d = 3, events = ['cagemate', 'recall', 'novel', 'acquisition'], plot = False)
PCA_matrix = pca_df
PCA_key = ['cagemate','recall', 'novel', 'acquisition']
equalize = 2
pre_window =2
post_window = 0
azim=45
elev=20
timebin = 50
color_id_dict = {'acquisition\n(exposure 1)': hex_2_rgb('#001424'), 'novel': hex_2_rgb('#15616F'), 
                'short-term familiar\n(exposure 2)': (1.0, 0.6862745098039216, 0.0), 'long-term familiar': hex_2_rgb('#792910')}

event_lengths = int(
            (equalize + pre_window + post_window) * 1000 / timebin
        )
event_end = int((equalize + pre_window) * 1000 / timebin)
pre_window = pre_window * 1000 / timebin
post_window = post_window * 1000 / timebin
colors = [hex_2_rgb('#792910'),  (1.0, 0.6862745098039216, 0.0), hex_2_rgb('#15616F'), hex_2_rgb('#001424')]
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
counter = 0 
for i in range(0, len(PCA_key)*event_lengths, event_lengths):
    event_label = PCA_key[counter]
    onset = int(i + pre_window - 1)
    end = int(i + event_end - 1)
    post = int(i + event_lengths - 1)
    ax.scatter(
        PCA_matrix[i : i + event_lengths, 0],
        PCA_matrix[i : i + event_lengths, 1],
        PCA_matrix[i : i + event_lengths, 2],
        label=event_label,
        s=5,
        color=colors[counter],
    )
    ax.scatter(
        PCA_matrix[i, 0],
        PCA_matrix[i, 1],
        PCA_matrix[i, 2],
        marker="s",
        s=100,
        color="w",
        edgecolors=colors[counter],
    )
    ax.scatter(
        PCA_matrix[onset, 0],
        PCA_matrix[onset, 1],
        PCA_matrix[onset, 2],
        marker="^",
        s=150,
        color="w",
        edgecolors=colors[counter],
    )
    ax.scatter(
        PCA_matrix[end, 0],
        PCA_matrix[end, 1],
        PCA_matrix[end, 2],
        marker="o",
        s=100,
        color="w",
        edgecolors=colors[counter],
    )
    counter += 1
ax.set_xlabel("\nPC1 (58% of variance)", fontsize = 14)
ax.set_ylabel("\nPC2 (24% of variance)", fontsize = 14)
ax.set_zlabel("\nPC3 (12% of variance)", fontsize = 14)
ax.tick_params(labelsize = 12)
ax.view_init(azim = 40, elev = 20
    )
ax.set_xlim(-20, 25)
ax.set_ylim(-15, 20)
ax.set_zlim(-10, 15)
id_labels = [plt.Line2D([0], [0], marker = 'o', color=color_id_dict[label], lw=0, label=label) for label in color_id_dict]
legend_labels = id_labels
ax.legend(loc="upper left", bbox_to_anchor=(1, .80), frameon = False, 
fontsize = 14, markerscale = 2, handles = legend_labels)
plt.title('Freely Moving', fontsize = 18, y = 1, x = .5)
plt.show()

#[0.58783251, 0.21168139, 0.12948744])

# Geodesic Distances 

In [ ]:
pca_analysis = spike.SpikeAnalysis_MultiRecording(phase2_collection, timebin = 100,ignore_freq = 0.5)
avg_geo_dis = pca_analysis.avg_geo_dist(2,2, 0.9,events = ['acquisition', 'recall', 'cagemate', 'novel'] )
avg_geo_dis_df = pd.DataFrame.from_dict(avg_geo_dis)
#acg_val = avg_geo_dis_df.mean()
#combined_index = avg_geo_dis_df.index.get_level_values("Outer") + avg_geo_dis_df.index.get_level_values("Inner")
#avg_geo_dis_df.index = combined_index
avg_geo_dis_df.columns.to_list()
avg_geo_dis_df.columns = ['acq vs cagemate', 'acq vs nov', 'acq vs recall', 'cage vs novel', 'cage vs recall', 'nov vs recall']
avg_vals = avg_geo_dis_df.mean()
avg_vals

## Plotting

In [ ]:
data =[[0, 773.5, 545.5, 1220.5],
       [773.5, 0, 753.9, 1234.5],
       [545.5, 753.9, 0, 1255.7],
       [1220.5, 1234.5, 1255.7, 0]]




plt.figure(figsize=(4,3))
col_pal = sns.dark_palette("#FFAF00", reverse=True, as_cmap=True)
heatmap = sns.heatmap(data, cmap=col_pal, annot=True, fmt = '.2f', vmin = 500)  # 'd' for integer format
heatmap.set_xticklabels(['Long-term', 'Short-term', 'Novel', 'Acquisition'], fontsize = 12, rotation = 45)
heatmap.set_yticklabels(['Long-term', 'Short-term', 'Novel', 'Acquisition'], fontsize = 12, rotation = 0)
heatmap.xaxis.set_ticks_position('top')
plt.title('Geodesic distances with 3 PCs (>90% variance)')

## Stats

In [ ]:
p_values = pd.DataFrame(index=avg_geo_dis_df.columns, columns=avg_geo_dis_df.columns)
for group1 in avg_geo_dis_df.columns:
    for group2 in avg_geo_dis_df.columns:
        if group1 != group2:
            t_stat, p_value = stats.ttest_ind(avg_geo_dis_df[group1], avg_geo_dis_df[group2])
            p_values.at[group1, group2] = p_value
p_values

In [ ]:
order_distances_df = pd.DataFrame.from_dict(order_distances)
avg_order_distances = order_distances_df.mean()
avg_order_distances

In [ ]:

data = [[0, 569, 777, 1148],
        [569, 0, 747, 1286],
        [777, 747, 0 , 1253],
        [1148, 1286, 1253, 0]]
plt.figure(figsize=(4,3))
col_pal = sns.dark_palette("#5BC", reverse=True, as_cmap=True)
heatmap = sns.heatmap(data, cmap=col_pal, annot=True, fmt = '.2f', vmin = 500)  # 'd' for integer format
heatmap.set_xticklabels(['Acquisition','Exposure 1', 'Exposure 2', 'Exposure 3'], fontsize = 12, rotation = 45)
heatmap.set_yticklabels(['Acquisition','Exposure 1', 'Exposure 2', 'Exposure 3'], fontsize = 12, rotation = 0)
heatmap.xaxis.set_ticks_position('top')
plt.title('Geo distances based on exposure order', fontsize = 16)
plt.show()

In [ ]:
average_values = geo_dis_df.mean()
sems = geo_dis_df.sem()
plt.bar(['Acquisition vs Cagemate', 'Acquisition vs Novel',
         'Acquisition vs Familiar', 'Cagemate vs Novel', 'Cagemate vs Familiar',
         'Novel vs Familiar'], average_values, yerr=sems, capsize=5)

# Label the x-axis and y-axis
plt.ylabel('Geodesic Distance')
plt.xticks(rotation = 45)
# Show the plot
plt.show()
